In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

# Cargar los datos
df = pd.read_csv('../../../sell-in.txt/sell-in.txt', sep='\t')
productosPredecir = pd.read_csv('C:/Users/Josvaldes/Documents/Maestria/Austral/2ano/Labo3/datasets/Proyecto/Labo3/Datasets/productos_a_predecir.txt', sep='\t')

# Convertir el periodo a formato datetime
df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m')

# Agregar los datos por periodo y product_id para obtener la serie temporal
ts = df.groupby(['periodo', 'product_id'])['tn'].sum().reset_index()

# Asegurarse de que las columnas tengan el mismo tipo y formato
ts['product_id'] = ts['product_id'].astype(int)
ts['periodo'] = pd.to_datetime(ts['periodo'])

# Agregar lags a tus datos
lags = 3  # Número de lags a incluir
for lag in range(1, lags + 1):
    ts[f'tn_lag_{lag}'] = ts.groupby('product_id')['tn'].shift(lag)

# Eliminar filas con valores nulos resultantes de los lags
ts.dropna(inplace=True)

# Obtener la lista de productos únicos a predecir
product_ids = productosPredecir['product_id'].unique()

# Crear diccionario para almacenar resultados y métricas
results = []
metrics = []

# Entrenar y predecir por cada product_id individualmente
for product_id in tqdm(product_ids, desc="Predicting with LGBM"):
    # Filtrar datos para el product_id actual
    product_data = ts[ts['product_id'] == product_id].copy()
    
    if len(product_data) > lags:  # Verificar si hay suficientes datos para entrenar
        # Normalizar los datos del product_id actual
        scaler = StandardScaler()
        product_data[['tn', f'tn_lag_1', f'tn_lag_2', f'tn_lag_3']] = scaler.fit_transform(
            product_data[['tn', f'tn_lag_1', f'tn_lag_2', f'tn_lag_3']]
        )

        # Crear características adicionales si es necesario (Ejemplo: características temporales)
        product_data['year'] = product_data['periodo'].dt.year
        product_data['month'] = product_data['periodo'].dt.month

        # Dividir los datos en conjunto de entrenamiento (hasta noviembre de 2019) y prueba (diciembre de 2019)
        train_data = product_data[product_data['periodo'] < '2019-12-01']
        test_data = product_data[product_data['periodo'] == '2019-12-01']

        # Crear conjunto de entrenamiento y objetivo
        X_train = train_data[['year', 'month'] + [f'tn_lag_{lag}' for lag in range(1, lags + 1)]]
        y_train = train_data['tn']
        X_test = test_data[['year', 'month'] + [f'tn_lag_{lag}' for lag in range(1, lags + 1)]]
        y_test = test_data['tn']

        # Crear y entrenar el modelo LightGBM con Random Forest
        train_data_lgb = lgb.Dataset(X_train, label=y_train)
        valid_data_lgb = lgb.Dataset(X_test, label=y_test, reference=train_data_lgb)

        params = {
            'objective': 'regression',
            'boosting': 'rf',  # Utilizar Random Forest para generar promedios en el árbol
            'num_iterations': 1000,  # Ajustar la cantidad de iteraciones según sea necesario
            'learning_rate': 0.009779220990172477,
            'num_leaves': 59,
            'bagging_freq': 4,
            'bagging_fraction': 0.798729860021949,
            'verbose': -1
        }

        # Entrenar el modelo LightGBM
        model = lgb.train(params, train_data_lgb, valid_sets=[train_data_lgb, valid_data_lgb], valid_names=['train', 'valid'])

        # Predicción para el último periodo disponible + 2 meses (febrero de 2020)
        last_period = product_data['periodo'].max()
        next_period = last_period + pd.DateOffset(months=2)
        next_data = pd.DataFrame({
            'year': [next_period.year],
            'month': [next_period.month],
            'tn_lag_1': [product_data['tn'].iloc[-1]],
            'tn_lag_2': [product_data['tn_lag_1'].iloc[-1]],
            'tn_lag_3': [product_data['tn_lag_2'].iloc[-1]]
        })

        pred = model.predict(next_data)[0]

        # Desnormalizar la predicción
        descaled_pred = scaler.inverse_transform([[pred, 0, 0, 0]])[0, 0]

        results.append({
            'product_id': product_id,
            'predicted_tn': max(descaled_pred, 0)  # Ajustar predicciones negativas a cero
        })

        # Evaluar el modelo en el conjunto de prueba
        y_pred = model.predict(X_test)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        metrics.append({
            'product_id': product_id,
            'rmse': rmse,
            'mae': mae,
            'r2': r2
        })
    else:
        # Predicción por defecto para productos con datos insuficientes
        results.append({
            'product_id': product_id,
            'predicted_tn': 0  # Ajustar según sea necesario
        })
        metrics.append({
            'product_id': product_id,
            'rmse': np.nan,
            'mae': np.nan,
            'r2': np.nan
        })

# Convertir los resultados a un DataFrame
results_df = pd.DataFrame(results)

# Asegurarse de que el DataFrame resultante tiene las columnas product_id y predicted_tn
results_df = results_df[['product_id', 'predicted_tn']]

# Exportar a un archivo CSV con las columnas product_id y predicted_tn
results_df.to_csv('C:/Users/Josvaldes/Documents/Maestria/Austral/2ano/Labo3/datasets/Proyecto/Labo3/Predicciones/lgbm_predictionsv2_by3.csv', index=False)

print("Predicciones exportadas a 'lgbm_predictionsv2_by3.csv'")

# Convertir las métricas a un DataFrame y exportar en el formato requerido
metrics_df = pd.DataFrame(metrics)
metrics_df.to_csv('C:/Users/Josvaldes/Documents/Maestria/Austral/2ano/Labo3/datasets/Proyecto/Labo3/Predicciones/lgbm_metricsv2_by3.csv', index=False, columns=['product_id', 'rmse', 'mae', 'r2'])

print("Métricas exportadas a 'lgbm_metricsv2_by3.csv'")

Predicting with LGBM:   0%|          | 0/780 [00:00<?, ?it/s]c:\Users\Josvaldes\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\Users\Josvaldes\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\Josvaldes\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
Predicting with LGBM:   0%|          | 1/780 [00:02<25:58,  2.00s/it]c:\Users\Josvaldes\AppData\Local\Programs\Python\Python311\Lib\site-packag

Predicciones exportadas a 'lgbm_predictionsv2_by3.csv'
Métricas exportadas a 'lgbm_metricsv2_by3.csv'
